# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

In [8]:
def filter_spike_times(arr_2d, start, stop, padding=-1):
    """
    Filters a 2D array based on a start and stop condition and pads shorter arrays to match the length of the longest array.

    Parameters:
    arr_2d (numpy.ndarray): 2D array to be filtered and padded.
    start (int or float): Lower bound of the condition.
    stop (int or float): Upper bound of the condition.
    padding (int or float): Value to use for padding, defaults to -1.

    Returns:
    numpy.ndarray: A 2D array where each inner array has been filtered based on the condition and padded to match the length of the longest array.
    """

    # Filter each row in arr_2d based on the condition
    masked_data = [row[(row >= start) & (row < stop)] for row in arr_2d]

    # Determine the maximum length of arrays in the list
    max_length = max(len(arr) for arr in masked_data)

    # Pad each array in masked_data with the padding value so they all have the same length (max_length)
    padded_arrays = [np.concatenate([x, np.full([max_length - len(x)], padding)]) for x in masked_data]

    # Convert the list of arrays to a 2D numpy array
    padded_arrays = np.array(padded_arrays)

    return padded_arrays

## Inputs & Data

Explanation of each input and where it comes from.

In [9]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("/nancy/user/riwata/projects/reward_comp_ext/results/2024_11_13_extract_lfp/proc/rce_pilot_3_alone_comp_01_lfp_traces_and_frames.pkl")
# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [10]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,vHPC_lfp_trace,BLA_lfp_trace,LH_lfp_trace,MD_lfp_trace,mPFC_lfp_trace
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ...","[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[1332, 1384, 2770, 4156, 5267, 5542, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ...","[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9...","[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[1332, 1384, 2770, 4156, 5267, 5542, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9...","[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...",

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [11]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_camp"

In [12]:
FULL_LFP_TRACES_PKL = "{}_02_lfp_traces_and_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [13]:
ALL_PHY_DIR

['/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240411_155157_comp_novel_subj_3-4_t4b4_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240318_143819_long_comp_subj_3-3_t6b6_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240408_155308_comp_novel_subj_5-3_t3b3_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240317_151922_long_comp_subj_3-1_t6b6_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240323_122227_alone_comp_subj_5-2_t6b6_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240407_152222_comp_novel_subj_3-3_t3b3_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240319_160457_long_comp_subj_4-2_t5b5_merged.rec',
 '/scratch/back_up/reward_competition_extention/final_proc/phy_curation/rce3/20240320_

In [14]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

In [15]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,0,23,6,89,8.217264,0,0,0.0,5.332289,mua,15683,0,1.0
1,2,99,28,223,9.195498,5,0,100.0,9.547664,mua,28081,0,4.0
2,5,427,211,784,11.845968,10,0,200.0,13.011289,mua,38268,0,7.0
3,6,3,3,4,24.675013,11,0,220.0,3.538086,good,10406,0,8.0
4,10,11,1,36,10.264018,15,0,300.0,8.493650,mua,24981,0,12.0


- Combining all the unit info dataframes and adding the recording name

In [16]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [17]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,1_5_ms_isi
0,20240411_155157_comp_novel_subj_3-4_t4b4_merged,2,6,83,7.283400,4,0.0,80.0,5.257556,mua,16779,0,3.0,NaN
1,20240411_155157_comp_novel_subj_3-4_t4b4_merged,5,0,2,10.911428,7,0.0,140.0,1.218898,good,3890,0,6.0,NaN
2,20240411_155157_comp_novel_subj_3-4_t4b4_merged,11,2,37,8.095570,13,0.0,260.0,4.040225,mua,12894,0,12.0,NaN
3,20240411_155157_comp_novel_subj_3-4_t4b4_merged,13,0,1,25.858170,16,0.0,320.0,5.593772,good,17852,0,14.0,NaN
4,20240411_155157_comp_novel_subj_3-4_t4b4_merged,14,0,0,23.164261,17,0.0,340.0,0.173278,good,553,0,15.0,NaN


- Filtering for the good units

In [18]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [19]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,1_5_ms_isi
0,20240411_155157_comp_novel_subj_3-4_t4b4_merged,5,0,2,10.911428,7,0.0,140.0,1.218898,good,3890,0,6.0,NaN
1,20240411_155157_comp_novel_subj_3-4_t4b4_merged,13,0,1,25.858170,16,0.0,320.0,5.593772,good,17852,0,14.0,NaN
2,20240411_155157_comp_novel_subj_3-4_t4b4_merged,14,0,0,23.164261,17,0.0,340.0,0.173278,good,553,0,15.0,NaN
3,20240411_155157_comp_novel_subj_3-4_t4b4_merged,21,1,2,18.076609,19,0.0,380.0,5.296724,good,16904,0,23.0,NaN
4,20240411_155157_comp_novel_subj_3-4_t4b4_merged,22,0,0,19.231091,19,0.0,380.0,1.620289,good,5171,0,24.0,NaN


In [20]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [21]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [22]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([84, 11, 71, ...,  2, 92, 60], dtype=int32)

In [23]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(485667,)

- The times that all the spikes happened

In [24]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [25]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[     272],
       [     322],
       [     415],
       ...,
       [63827793],
       [63827836],
       [63828089]])

In [26]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(485667, 1)

### Combining everything into a dataframe

In [27]:
recording_to_spike_df = {}

for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

In [28]:
cluster_info_df.head()

,cluster_id,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,0,2,29,10.700995,0,0,0.0,2.619794,mua,8391,0,1
1,1,1,9,7.437864,0,0,0.0,1.531413,mua,4905,0,2
2,2,2,8,7.615370,0,0,0.0,2.005980,mua,6425,0,3
3,3,0,1,25.458071,0,0,0.0,0.271627,mua,870,0,4
4,4,2,27,7.876468,4,0,80.0,3.527090,mua,11297,0,5


In [29]:
spike_times

array([[     355],
       [     426],
       [    1198],
       ...,
       [64058231],
       [64058262],
       [64058403]])

In [30]:
spike_clusters

array([18, 12, 18, ..., 39, 39, 52], dtype=int32)

- Combining the spike time df for all recordings

In [31]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [32]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [33]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi,1_5_ms_isi
0,33,532,33,0,0,19.129400,23,0.0,460.0,0.473772,good,1512,0,35.0,20240411_155157_comp_novel_subj_3-4_t4b4_merged,NaN,NaN,NaN
1,13,1272,13,0,1,25.858170,16,0.0,320.0,5.593772,good,17852,0,14.0,20240411_155157_comp_novel_subj_3-4_t4b4_merged,NaN,NaN,NaN
2,92,2228,92,3,12,11.571054,27,NaN,540.0,4.830158,good,15415,0,NaN,20240411_155157_comp_novel_subj_3-4_t4b4_merged,NaN,NaN,NaN
3,63,2265,63,0,0,26.958054,13,NaN,260.0,0.817508,good,2609,0,NaN,20240411_155157_comp_novel_subj_3-4_t4b4_merged,NaN,NaN,NaN
4,87,2384,87,0,0,21.646833,21,NaN,420.0,1.219525,good,3892,0,NaN,20240411_155157_comp_novel_subj_3-4_t4b4_merged,NaN,NaN,NaN


In [34]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi,1_5_ms_isi
7612494,9,64056268,9,0,1,46.368992,8,0.0,160.0,1.095562,good,3509,0,10.0,20240410_152009_comp_novel_subj_4-3_t6b6_merged,17686.0,0.88430,NaN
7612495,52,64056649,52,0,6,17.425798,19,0.0,380.0,4.665737,good,14944,0,26.0,20240410_152009_comp_novel_subj_4-3_t6b6_merged,2843.0,0.14215,NaN
7612496,13,64057879,13,0,0,25.287748,14,0.0,280.0,2.247634,good,7199,0,15.0,20240410_152009_comp_novel_subj_4-3_t6b6_merged,3751.0,0.18755,NaN
7612497,13,64058014,13,0,0,25.287748,14,0.0,280.0,2.247634,good,7199,0,15.0,20240410_152009_comp_novel_subj_4-3_t6b6_merged,135.0,0.00675,NaN
7612498,52,64058403,52,0,6,17.425798,19,0.0,380.0,4.665737,good,14944,0,26.0,20240410_152009_comp_novel_subj_4-3_t6b6_merged,1754.0,0.08770,NaN


## Grouping all the neurons by recording

In [35]:
# Grouping all spike times by neuron and recording
grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: np.array(x)).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)

In [36]:
grouped_df.head()

,spike_clusters,recording_name,spike_times
0,28,20240317_151922_long_comp_subj_3-1_t6b6_merged,"[1114, 3560, 6526, 8227, 9818, 13111, 14728, 1..."
1,60,20240317_151922_long_comp_subj_3-1_t6b6_merged,"[9207, 10264, 11524, 11872, 14268, 22414, 3833..."
2,92,20240317_151922_long_comp_subj_3-1_t6b6_merged,"[20065, 108369, 117274, 127160, 133418, 152238..."
3,111,20240317_151922_long_comp_subj_3-1_t6b6_merged,"[18919, 22489, 34070, 34749, 37126, 57953, 787..."
4,121,20240317_151922_long_comp_subj_3-1_t6b6_merged,"[1129, 5706, 8081, 8344, 11627, 14332, 20891, ..."


In [37]:
max_number_of_spikes = all_spike_time_df["n_spikes"].max()

In [38]:
max_number_of_spikes

126385

In [39]:
grouped_df["spike_times"] = grouped_df["spike_times"].apply(lambda x: np.concatenate([x, np.full([max_number_of_spikes - x.shape[0]], np.nan)]))

In [40]:
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: np.array(list(x))}).reset_index()

In [41]:
LFP_SPECTRAL_DF = pd.merge(LFP_SPECTRAL_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

In [42]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,box_2_port_entry_timestamps,lfp_timestamps,vHPC_lfp_trace,BLA_lfp_trace,LH_lfp_trace,MD_lfp_trace,mPFC_lfp_trace,recording_name,spike_clusters,spike_times
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ...","[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ...",20240320_142408_alone_comp_subj_3-1_t6b6_merged,"[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144...","[[203.0, 1577.0, 4602.0, 7261.0, 7971.0, 10653..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ...","[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ...",20240320_142408_alone_comp_subj_3-1_t6b6_merged,"[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144...","[[203.0, 1577.0, 4602.0, 7261.0, 7971.0, 10653..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9...","[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ...",20240320_142408_alone_comp_subj_3-3_t5b5_merged,"[45, 46, 60, 67, 109, 110, 124, 136, 146, 148]","[[7700.0, 31351.0, 32696.0, 34739.0, 35395.0, ..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9...","[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ...",20240320_142408_alone_comp_subj_3-3_t5b5_merged,"[45, 46, 60, 67, 109, 110, 124, 136, 146, 148]","[[7700.0, 31351.0, 32696.0, 34739.0, 35395.0, ..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132], [145, 175

## Calculating the firing rates

In [43]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'notes', 'session_path',
       'recording', 'current_subject', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace', 'mPFC_lfp_trace',
       'recording_name', 'spike_clusters', 'spike_times'],
      dtype='object')

In [44]:
cluster_info_df.head()

,cluster_id,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,0,2,29,10.700995,0,0,0.0,2.619794,mua,8391,0,1
1,1,1,9,7.437864,0,0,0.0,1.531413,mua,4905,0,2
2,2,2,8,7.615370,0,0,0.0,2.005980,mua,6425,0,3
3,3,0,1,25.458071,0,0,0.0,0.271627,mua,870,0,4
4,4,2,27,7.876468,4,0,80.0,3.527090,mua,11297,0,5


In [45]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,box_2_port_entry_timestamps,lfp_timestamps,vHPC_lfp_trace,BLA_lfp_trace,LH_lfp_trace,MD_lfp_trace,mPFC_lfp_trace,recording_name,spike_clusters,spike_times
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ...","[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ...",20240320_142408_alone_comp_subj_3-1_t6b6_merged,"[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144...","[[203.0, 1577.0, 4602.0, 7261.0, 7971.0, 10653..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ...","[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ...",20240320_142408_alone_comp_subj_3-1_t6b6_merged,"[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144...","[[203.0, 1577.0, 4602.0, 7261.0, 7971.0, 10653..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9...","[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ...",20240320_142408_alone_comp_subj_3-3_t5b5_merged,"[45, 46, 60, 67, 109, 110, 124, 136, 146, 148]","[[7700.0, 31351.0, 32696.0, 34739.0, 35395.0, ..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9...","[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ...",20240320_142408_alone_comp_subj_3-3_t5b5_merged,"[45, 46, 60, 67, 109, 110, 124, 136, 146, 148]","[[7700.0, 31351.0, 32696.0, 34739.0, 35395.0, ..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132], [145, 175

In [46]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times[~np.isnan(times)]), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)


In [47]:
LFP_SPECTRAL_DF["neuron_average_timestamps"] = LFP_SPECTRAL_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0][~np.isnan(x["spike_times"][0])], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [48]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [49]:
LFP_SPECTRAL_DF.iloc[:5,:10]

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79], [79, 186], [188, 191], [191, 268], [...",20240320_171038_alone_comp_subj_4-2_and_4-3.1....,NaN,/scratch/back_up/reward_competition_extention/...,20240320_171038_alone_comp_subj_4-2_t6b6_merged,4.2


In [50]:
LFP_SPECTRAL_DF.iloc[:5,10:20]

,subject,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,vHPC_lfp_trace
0,3.1,"[3.1, 3.3]",1830734,65425515,"[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ..."
1,3.1,"[3.1, 3.3]",1830734,65425515,"[1332, 1384, 2770, 4156, 5267, 5542, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[337, 494, 488, 398, 579, 592, 501, 539, 512, ..."
2,3.3,"[3.1, 3.3]",1830734,65425515,"[1384, 2770, 4156, 4156, 5542, 6928, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9..."
3,3.3,"[3.1, 3.3]",1830734,65425515,"[1332, 1384, 2770, 4156, 5267, 5542, 6928, 831...","[[0, 75474], [1275889, 1475891], [3275911, 347...","[[0, 75474], [153075, 158874], [252873, 306276...","[[0, 75474], [162074, 226073], [255076, 258076...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-516, -528, -666, -850, -982, -1047, -959, -9..."
4,4.2,"[4.2, 4.3]",2067718,66452444,"[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[[0, 79744], [85741, 92941], [93544, 134344], ...","[[0, 79744], [80141, 188142], [188942, 191545]...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-383, -582, -804, -996, -1365, -1694, -1860, ..."


In [51]:
LFP_SPECTRAL_DF.iloc[:5,20:30]

,BLA_lfp_trace,LH_lfp_trace,MD_lfp_trace,mPFC_lfp_trace,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,"[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ...",20240320_142408_alone_comp_subj_3-1_t6b6_merged,"[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144...","[[203.0, 1577.0, 4602.0, 7261.0, 7971.0, 10653...","[[2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,"[577, 737, 658, 382, 345, 176, -30, -61, -114,...","[587, 747, 643, 319, 332, 336, 284, 377, 435, ...","[762, 969, 941, 683, 739, 746, 661, 663, 653, ...","[373, 642, 608, 445, 716, 678, 548, 716, 776, ...",20240320_142408_alone_comp_subj_3-1_t6b6_merged,"[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144...","[[203.0, 1577.0, 4602.0, 7261.0, 7971.0, 10653...","[[2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
2,"[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ...",20240320_142408_alone_comp_subj_3-3_t5b5_merged,"[45, 46, 60, 67, 109, 110, 124, 136, 146, 148]","[[7700.0, 31351.0, 32696.0, 34739.0, 35395.0, ...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
3,"[-220, -135, -112, -164, -146, -126, -86, -69,...","[-230, -162, -155, -263, -320, -287, -197, -15...","[-143, -90, -49, -57, -55, -96, -159, -148, -1...","[-199, -103, 14, 58, 146, 332, 471, 381, 235, ...",20240320_142408_alone_comp_subj_3-3_t5b5_merged,"[45, 46, 60, 67, 109, 110, 124, 136, 146, 148]","[[7700.0, 31351.0, 32696.0, 34739.0, 35395.0, ...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
4,"[178, 104, 146, 246, 138, -7, 48, 325, 463, 46...","[152, 201, 236, 195, 13, -249, -238, 10, 191, ...","[-13, -9, 202, 348, 68, -189, -136, 47, 199, 3...","[4, -240, -358, -373, -650, -828, -870, -769, ...",20240320_171038_alone_comp_subj_4-2_t6b6_merged,"[5, 9, 33, 40, 42, 56]","[[4011.0, 66707.0, 149318.0, 161401.0, 166891....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."


In [52]:
LFP_SPECTRAL_DF.iloc[:5, 30:]

""
0
1
2
3
4


In [53]:
LFP_SPECTRAL_DF["neuron_average_timestamps"].iloc[0].shape

(31797,)

In [54]:
LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0].shape

(15, 31797)

In [55]:
np.max(LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0])

12.0

In [56]:
LFP_SPECTRAL_DF.to_pickle(os.path.join("./proc", FULL_LFP_TRACES_PKL))

In [57]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'notes', 'session_path',
       'recording', 'current_subject', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'vHPC_lfp_trace',
       'BLA_lfp_trace', 'LH_lfp_trace', 'MD_lfp_trace', 'mPFC_lfp_trace',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps'],
      dtype='object')

In [58]:
LFP_SPECTRAL_DF["spike_clusters"].iloc[0]

[4, 9, 13, 18, 20, 93, 115, 117, 123, 143, 144, 149, 150, 154, 190]

In [59]:
LFP_SPECTRAL_DF["spike_times"].iloc[0].shape

(15, 126385)

In [60]:
raise ValueError("")

ValueError: 